In [1]:
from utils import load_images, to_optimize
import numpy as np
import os

In [2]:
# create a logbook to record the results
import logging

# Create a logger
logger = logging.getLogger('training_logger')
logger.setLevel(logging.INFO)

# Create a file handler
handler = logging.FileHandler('training.log')
handler.setLevel(logging.INFO)

# Create a logging format
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

# Add the handler to the logger
logger.addHandler(handler)


In [3]:
cwd = os.getcwd()
print(cwd)

num, files = load_images(cwd)
print('number of files: ', num)

files = list(files)

/Users/vanple/Desktop/ucla/_Courses_/3D-vis-reasoning/stage2/stage2_rectification
number of files:  11


In [4]:
# grid search for the best parameters and methods:
'''
    1. feature detection method:
        - SIFT
        - SURF # not working, patented
        - ORB
    2. feature matching method:
        - Brute Force
        - FLANN
    3. feature matching ratio:
        - 0.7
        - 0.8
        - 0.9
    
'''
param_grid = {
    'f_detector': ['SIFT', 'ORB'],
    'nfeatures': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
    'contrastThreshold': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06],
    'f_matcher': ['BruteForce', 'FLANN'],
    'f_ratio': [0.7, 0.8, 0.9]
}

## perform the parallel grid search

In [ ]:
# grid search in parallel
import concurrent.futures
import itertools

# create a list of all the combinations
all_combinations = list(itertools.product(*param_grid.values()))

# perform the grid search in parallel
with concurrent.futures.ProcessPoolExecutor() as executor:
    futures = [executor.submit(to_optimize, params, files, num) for params in all_combinations]
    for i, future in enumerate(concurrent.futures.as_completed(futures)):
        logger.info('finished {}/{} tasks'.format(i+1, len(all_combinations)))
        logger.info('params: {}'.format(all_combinations[i]))
        logger.info('result: {}'.format(future.result()))
        logger.info('----------------------------------')

In [7]:
# the best parameters are:
best_result = max(futures, key=lambda x: x.result())
print('best result: ', best_result.result())
print('best params: ', all_combinations[futures.index(best_result)])

best result:  31.04
best params:  SIFT 700 0.05 BruteForce 0.8


Side Notes: SURF might be better than SIFT? Since it is patented, maybe companies use it for better performance?